In [3]:
import requests


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

# import sys
import requests
url = 'https://www.wikidata.org/w/api.php'

def search_entities(name, url):
    """
    Get the entity id, using a name, by calling the Wiki API. 
    The output is a list of dict of dictionaries corresponding to the number of
    occurances of the given name 
    """
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': name
    }
    response = requests.get(url, params=params)
    return response.json()['search']



In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import sys

# endpoint_url = "https://query.wikidata.org/sparql"

# query = """
# SELECT ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel
# WHERE 
# {
#   ?item wdt:P106 ?occupation .
#   ?item wdt:P21 ?gender .
#   ?item wdt:P18 ?image .
#   ?item wdt:P569 ?bday .
#   ?item wdt:P27 ?citizenship
#   FILTER(?item = wd:Q37079)
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
# }
# """


def get_results(endpoint_url, query):
    """
    Get's the information from endpoint (wikidata), by submitting a SPAQL query,
    returning s corresponding JSON file
    """
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


# results = get_results(endpoint_url, query)

# for result in results["results"]["bindings"]:
#     print(result)

In [5]:
import pandas as pd
# Get all the names and entity_id
data = pd.read_csv("../raw_data/list_act.csv") #read current names from the list_act csv

In [4]:
#new dictionary to ho
new_file_dict = {
    "name":[],
    "wiki_id":[]
}
for name in data["name"]:
    res = search_entities(name,url=url)[0]["id"]
    new_file_dict["name"].append(name)
    new_file_dict["wiki_id"].append(res)
   

In [5]:
df_act = pd.DataFrame(new_file_dict)


In [6]:
df_act.to_csv("../raw_data/celebrity_list.csv")

In [6]:
df_act = pd.read_csv("../raw_data/celebrity_list.csv")
import sys
sys.path.insert(0, '../')
import pandas as pd


In [ ]:
# Create a new df with all the name taken from the df having wiki_id
from logic import WikiDataQueryResults as wdr


new_df = pd.DataFrame(columns=['itemLabel','item', 'image', 'occupationLabel','genderLabel', 'bdayLabel','citizenshipLabel','dodLabel'])

for wiki_id in df_act["wiki_id"]:

    query = f"""
    SELECT ?itemLabel ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel ?dodLabel
    WHERE 
    {{
    ?item wdt:P106 ?occupation .
    ?item wdt:P21 ?gender .
    ?item wdt:P18 ?image .
    ?item wdt:P569 ?bday .
    ?item wdt:P27 ?citizenship .
    ?item wdt:P570 ?dod .
    FILTER(?item = wd:{wiki_id})
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    data_extracter = wdr(query)
    df = data_extracter.load_as_dataframe()
    new_df = pd.concat([new_df,df[0:1]])
new_df.to_csv("../raw_data/metafile.csv") #write to csv file

In [9]:
new_df.to_csv("../raw_data/metafile.csv")

In [7]:
# Create a new df with all the name taken from the df having wiki_id
from logic import WikiDataQueryResults as wdr


new_df = pd.DataFrame(columns=['itemLabel','item', 'image', 'occupationLabel','genderLabel', 'bdayLabel','citizenshipLabel'])

for wiki_id in df_act["wiki_id"]:

    query = f"""
    SELECT ?itemLabel ?item ?occupationLabel ?image ?genderLabel ?bdayLabel ?citizenshipLabel
    WHERE 
    {{
    ?item wdt:P106 ?occupation .
    ?item wdt:P21 ?gender .
    ?item wdt:P18 ?image .
    ?item wdt:P569 ?bday .
    ?item wdt:P27 ?citizenship .
    FILTER(?item = wd:{wiki_id})
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """

    data_extracter = wdr(query)
    df = data_extracter.load_as_dataframe()
    new_df = pd.concat([new_df,df])
new_df.to_csv("../raw_data/metafile1.csv") #write to csv file

In [18]:
df_test = pd.read_csv("../raw_data/metafile1.csv")


In [10]:
df_test.columns

Index(['Unnamed: 0', 'itemLabel', 'item', 'image', 'occupationLabel',
       'genderLabel', 'bdayLabel', 'citizenshipLabel'],
      dtype='object')

In [19]:
df_test = df_test.groupby(['occupationLabel','itemLabel', ],as_index=False).agg({'itemLabel' : 'first', 'genderLabel':'first', 'bdayLabel':'first', 'citizenshipLabel':'first'})

In [21]:
df_test[df_test["itemLabel"]=="Cate Blanchett"]

,occupationLabel,itemLabel,genderLabel,bdayLabel,citizenshipLabel
29,actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
368,film actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
543,film director,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
631,film producer,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
732,international forum participant,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
987,stage actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
1259,television producer,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America
1310,voice actor,Cate Blanchett,female,1969-05-14T00:00:00Z,United States of America


In [16]:
df_test = df_test.groupby('itemLabel', as_index=False).agg({'itemLabel' : 'first', 'occupationLabel' : ', '.join,'genderLabel':'first', 'bdayLabel':'first', 'citizenshipLabel':'first'})

In [17]:
df_test.to_csv("../raw_data/metafile3.csv")

In [206]:
import os
import unicodedata

user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1]) #define agent
headers = {'User-Agent': user_agent}

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

# Iterate over all celebrities in df and get image and save to output_wiki
for idx, row in new_df.iterrows():
    img_url = new_df["image"].iloc[idx]
    # person_name = new_df["itemLabel"].iloc[idx]
    person_name = "_".join(strip_accents(new_df["itemLabel"].iloc[idx].lower()).split())+".jpg"
    target_path = os.path.join(os.getcwd(),"raw_data","output_wiki",person_name)
    target_image = target_path
    response = requests.get(img_url,stream=True,headers=headers)

    if response.status_code:
        fp = open(target_image, 'wb')
        fp.write(response.content)
        fp.close()
